In [2]:
import pandas as pd
import unidecode

# !pip install fuzzywuzzy
# !pip install p
from fuzzywuzzy import fuzz


ModuleNotFoundError: No module named 'fuzzywuzzy'

## 1. Merge past donors dataset (K4K sources) with donor organizations (PDF source)
We will use the company name in both sources to try and match companies. We will use fuzzy matching so we can detect cases with similar strings (its likely names will have different versions in the sources)

In [3]:
df_donors = pd.read_pickle('../Data/past_donors_clean.pickle')
df_orgs = pd.read_excel('../Data/Organizations.xlsx')

In [8]:
def stdr_names(series_original):
    '''Clean company names. To be used in the different files so names are likelier to be matched'''
    series = series_original.copy()
    series = series.astype(str)
    
    series = series.str.upper()
    series = series.str.strip()

    series = series.replace(r'\s+', ' ', regex=True)
    series = series.str.replace(r'[^\w\s]+', '', regex=True)


    series = series.apply(lambda x: unidecode.unidecode(x))

    series = series.str.replace(' QUEBEC ', '')
    series = series.str.replace('CANADA', '')
    series = series.str.replace(' MONTREAL ', '')
    series = series.str.replace('MONTREAL ', '')
    series = series.str.replace(' MONTREAL', '')
    series = series.str.replace(' INC', '')
    series = series.str.replace(' INC ', '')
    series = series.str.replace(' CIE ', '')
    series = series.str.replace(' CIE', '')
    series = series.str.replace(' LTEE', '')
    series = series.str.replace('CORPORATION', '')
    series = series.str.replace('INTERNATIONAL', '')


    return series


In [5]:
df_donors = df_donors[ df_donors.company.notna() ]

In [6]:
df_orgs.NAME = stdr_names(df_orgs.NAME)

df_donors.company = df_donors.company.astype(str)
df_donors.company  = stdr_names(df_donors.company)

In [7]:
pd.merge(df_orgs.NAME.drop_duplicates(), df_donors.company.drop_duplicates(), left_on='NAME', right_on='company')

# 18 matches by exact matching

,NAME,company
0,ARCHAMBAULT,ARCHAMBAULT
1,AVERNA,AVERNA
2,CANADIAN TIRE,CANADIAN TIRE
3,CENTRES DENTAIRES LAPOINTE,CENTRES DENTAIRES LAPOINTE
4,CGI,CGI
5,CORBEC,CORBEC
6,DORFIN,DORFIN
7,ENERGIE CARDIO,ENERGIE CARDIO
8,FIERA CAPITAL,FIERA CAPITAL
9,FUTURE SHOP,FUTURE SHOP


In [8]:
# https://www.datacamp.com/tutorial/fuzzy-string-python


# this for-loop calculates 4 kinds of similarity score, which we will then use to sort the table and manually review the most similar matches to confirm (based on K4K feedback)

def match_comp_names(orgs, donors):

    # orgs = orgs.NAME.drop_duplicates().to_list()
    # donors = donors.company.drop_duplicates().to_list()

    d = 0
    j = 0
    rows = []

    for org in orgs:
        for donor in donors:
            Str1 = org
            Str2 = donor
            Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
            Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
            Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
            Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)

            row = [org, donor, Ratio, Partial_Ratio, Token_Sort_Ratio, Token_Set_Ratio]

            rows.append(row)
    
    return rows

In [9]:
rows = match_comp_names(df_orgs.NAME.drop_duplicates().to_list(), df_donors.company.drop_duplicates().to_list())
df_sim = pd.DataFrame(rows, columns = ['org', 'donor', 'Ratio', 'Partial_Ratio', 'Token_Sort_Ratio', 'Token_Set_Ratio'] )
df_sim.to_pickle('similarity_score.pkl')


In [10]:
score = 'Ratio'
df_ratios = df_sim.groupby('org', as_index=False)[score].max()
#df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(30)

In [11]:
score = 'Token_Set_Ratio'

df_ratios = df_sim.groupby('donor', as_index=False)[score].max()
#df_token_set_ratio = df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False)

In [12]:
score = 'Partial_Ratio'
#df_ratios = df_sim.groupby('org', as_index=False)[score].max()
#df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(20)

In [13]:


score = 'Token_Sort_Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(20)

,org,Token_Sort_Ratio,donor
1317,KRUGER,100,KRUGER
846,CORBEC,100,CORBEC
1772,ROYAL LEPAGE,100,ROYAL LEPAGE
1329,LABORATOIRES CHARLES RIVER,100,LABORATOIRES CHARLES RIVER
881,DELMAR,100,DELMAR
1061,FUTURE SHOP,100,FUTURE SHOP
1278,JEAN COUTU,100,JEAN COUTU
613,BDO,100,BDO
733,CANADIAN TIRE,100,CANADIAN TIRE
1832,SNC LAVALIN,100,SNC LAVALIN


In [14]:
# we sort the values and export them to CSV so we can review in excel

th = 70
df_filt = df_sim.query(f''' Ratio > {th} |  Token_Sort_Ratio > {th}  | Token_Set_Ratio > {th}  ''' ).reset_index(drop=True)
df_filt.shape
df_filt.sort_values(['Token_Set_Ratio', 'Ratio'], ascending=False).to_clipboard(index=None)

## 2. Confirm matches based on reviewed excel
After manually confirming the fuzzy matching scores, we will produces the final merged dataset

In [5]:
import pandas as pd

In [9]:
df_donors = pd.read_csv('../Data/past_donors_clean.csv', parse_dates=[1])
df_orgs = pd.read_excel('../Data/Organizations.xlsx')

# now we read the matches after reviews them and this will become our key to match both sources
df_matches = pd.read_excel('../Data/matched_orgs_k4kreview.xlsx')

df_donors.company = stdr_names(df_donors.company)
df_orgs.NAME = stdr_names(df_orgs.NAME)

AttributeError: 'DataFrame' object has no attribute 'NAME'

In [ ]:
df_matches = df_matches[df_matches.Match == 1] # keeping only matches
df_matches = df_matches.iloc[:, 0:2]
df_matches.columns = ['ORGANIZATION', 'DONOR']

In [29]:
# some past donors have been matched with more than 1 different company in the PDF source, so we need to select and keep only one so the analysis is consistent.
donor_q = df_matches.DONOR.value_counts()
organization_q = df_matches.ORGANIZATION.value_counts()
df1 = df_matches[df_matches.DONOR.isin(donor_q[donor_q > 1].index)].sort_values('DONOR').reset_index(drop=True)
ixs = [1, 2,5,6,10,11,15,16,18, 21,22] 
df1 = df1.iloc[ixs,:].reset_index(drop=True)

In [30]:
# now we concatenate the matches into a single df
df_matches = pd.concat([ df_matches[df_matches.DONOR.isin(donor_q[donor_q <= 1].index)].sort_values('DONOR').reset_index(drop=True),
            df1 ] )

In [31]:
df_donors_match = df_donors.merge(df_matches, how = 'inner', left_on = 'company', right_on = 'DONOR')

In [32]:
df_donors_match = df_donors_match.merge(df_orgs, how = 'inner', left_on = 'ORGANIZATION', right_on = 'NAME')

In [33]:
df_donors_match = df_donors_match.drop_duplicates().reset_index(drop=True)


In [34]:
df_donors_match.shape

# after matching past donors and organizations PDF, we have 177 rows of data and 32 columns. We should add additiional attributes of the companies that have foundations

(177, 32)

## 3. Adding foundation attributes

In [35]:
df_fonds = pd.read_excel('Foundations.xlsx')

## add foundations dataset attributes

In [36]:
df = df_donors_match.merge(df_fonds, how='left', on='ID', suffixes=('', '_FND'))
df.shape

(177, 53)

In [37]:
df

,company,added_modified,dollar_amount_float,moneyprize_received,donation_details,year,ORGANIZATION,DONOR,ID,2E CONTACT POUR,...,LANGUE_FND,LIMITES GEOG_FND,N DE TEL_FND,NAME_FND,OU,POSTE_FND,PROJETS PRIVILEGIES,TEL,TOTAL ANNUEL,WEB
0,VIA,2010-01-19,0.0,NaN,trip for 4 Montreal/Windsor $2600 value,2010,VIA RAIL,VIA,1312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VIA,2011-01-01,0.0,Yes,trip for 4 Montreal/Windsor $2600 value,2011,VIA RAIL,VIA,1312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VIA,2012-01-09,0.0,Yes,trip for 4 Montreal/Windsor $2251 value,2012,VIA RAIL,VIA,1312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VIA,2012-06-12,0.0,Yes,trip for 4 Montreal/Windsor $2251 value,2013,VIA RAIL,VIA,1312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VIA,2014-03-04,0.0,Yes,Silver,2014,VIA RAIL,VIA,1312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,JEAN COUTU,2018-03-07,0.0,Yes,2 gift certificates,2018,JEAN COUTU,JEAN COUTU,677,"CommanditeMadame Raphaële Charette, Directeure...",...,F,NaN,514-527-4510,"Coutu,",NaN,NaN,Coopération internationale. Projets spéciaux.,514-527-4917,8 524 975 $,NaN
173,CENTRES DENTAIRES LAPOINTE,2018-03-09,500.0,No,$500 donation,2018,CENTRES DENTAIRES LAPOINTE,CENTRES DENTAIRES LAPOINTE,259,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,CIBC,2018-03-22,200.0,Yes,$200 1/4 page ad,2018,BANQUE CIBC,CIBC,121,NaN,...,A,NaN,416-861-8023,"CIBC Children's Foundation,",NaN,NaN,Éducation. Fondations hospitalières. Fonds d'é...,416-861-3757,3 914 950 $,www.cibc.com/miracleday
175,UPS,2018-04-02,1000.0,Yes,$1000 donation,2018,UPS,UPS,1286,NaN,...,A,NaN,800-742-5877,"UPS Foundation,",http://www.community.ups.com/UPS+Foundation,NaN,Dons jumelés (avec les employés). Fonds d'équi...,NaN,93 500 000 $,www.community.ups.com


In [38]:
#obviously, most companies will have null values in their foundation attributes, but we can still keep the info for the ones that do

In [41]:
df.to_pickle('dataset.pickle')

In [39]:

df.isna().sum()

company                  0
added_modified           5
dollar_amount_float     11
moneyprize_received     63
donation_details        13
year                     0
ORGANIZATION             0
DONOR                    0
ID                       0
2E CONTACT POUR        174
ADDRESS                  0
AVIS                     6
CONTACT                 10
CONTRIBUTION             0
COURRIEL                39
DDD                      0
DOMAINE DINTERET         0
FAF                     55
FAX                     60
FILIALE DE             144
ISFOUNDATION             0
LANGUE                   0
LIMITES GEOG             1
N DE TEL                 0
NAME                     0
NBRE DE SUCC           177
NOMBRE DEMPLOYES        59
NOTE                    44
POSTE                  168
PRINCIP FILIALES       137
SECTEUR INDUSTRIEL       0
SITE WEB                 0
ACTIF                  148
ADDRESS_FND            143
AVIS_FND               147
CATEGORIE              144
CONTACT_FND            146
C